Inspired by this dude https://www.kaggle.com/anokas/simple-xgboost-starter-0-0655/code

Did simple Random forest regression

Note: Used features from data exploration
Did not fintune any parameters
Also imputed values for test set

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [3]:
#Loading data
train_df = pd.read_csv('../data/train_2016.csv')
prop_df = pd.read_csv('../data/properties_2016.csv')
sample_df = pd.read_csv('../data/sample_submission.csv')

C:\Users\bhedayat\Anaconda3\envs\py27\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Merging data
train_df = pd.merge(train_df, prop_df, on='parcelid', how='left')

In [14]:
#Inspired by last dude
cols_to_use = ['taxamount','heatingorsystemtypeid','yearbuilt','bedroomcnt','fullbathcnt','calculatedbathnbr','bathroomcnt',
               'calculatedfinishedsquarefeet','finishedsquarefeet12']
x_train = train_df[cols_to_use]
#Impute
mean_values = x_train.mean(axis=0)
x_train = x_train.fillna(mean_values, inplace=True)
y_train = train_df['logerror'].values
print(x_train.shape, y_train.shape)

((90811, 9), (90811L,))


C:\Users\bhedayat\Anaconda3\envs\py27\lib\site-packages\pandas\core\generic.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [16]:
#Split data
split = 80000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]

In [20]:
#Fit to training data
clf = RandomForestRegressor()
clf.fit(x_train.values,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [25]:
#Building test set

sample_df['parcelid'] = sample_df['ParcelId']
df_test = sample_df.merge(prop_df, on='parcelid', how='left')

In [27]:
x_test = df_test[cols_to_use]

#Had to effin impute
x_test = x_test.fillna(mean_values, inplace=True)


In [29]:
#Predict
p_test = clf.predict(x_test.values)

In [31]:
sub = pd.read_csv('../data/sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

In [35]:
#Writing to csv
sub.to_csv('../data/rf_starter.csv', index=False, float_format='%.4f')